In [1]:
import pandas as pd
from datasets import PcapDataset
from constants import PCAP_PATH

import torch.nn as nn
import torch.optim as optim

from torchvision import transforms
from torch.utils.data import DataLoader
import sys
import torch

In [2]:
PCAP_PATH = "../data/benign/weekday.pcap"
CSV_PATH  = "../data/benign/weekday.csv"
batch_size = 1

In [3]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size) -> None:
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        trx = x.reshape(1, 1, x.shape[-1]).to(torch.float)
        _, (hidden, _) = self.lstm(trx)  # Use the last hidden state for prediction
        out = self.linear(hidden[-1])
        return out
# Instantiate the model with specified dimensions
model = LSTM(input_size=235, hidden_size=64, num_layers=2, output_size=100)

In [5]:
# Load feature vectors from CSV file
features = pd.read_csv(CSV_PATH).iloc[:, :-2]
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

for epoch in range(100):
    # Create the dataset
    dataset = PcapDataset(pcap_file=PCAP_PATH, max_iterations=sys.maxsize)

    # Create the DataLoader
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, drop_last=True)
    for i, packets in enumerate(dataloader):
        # Define loss function and optimizer
        outputs = model(packets)
        loss = criterion(outputs, torch.from_numpy(features.iloc[i].to_numpy()).to(torch.float))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print training progress
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

/home/subrat/anaconda3/envs/panda/lib/python3.10/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([1, 100])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


IndexError: single positional indexer is out-of-bounds

In [46]:
features = pd.read_csv(CSV_PATH).iloc[:, :-2]
torch.from_numpy(features.iloc[1].to_numpy()).to(torch.float)

tensor([1.0000e+00, 9.6000e+01, 0.0000e+00, 1.0000e+00, 9.6000e+01, 0.0000e+00,
        1.0000e+00, 9.6000e+01, 0.0000e+00, 1.0000e+00, 9.6000e+01, 0.0000e+00,
        1.0000e+00, 9.6000e+01, 0.0000e+00, 1.0000e+00, 9.6000e+01, 0.0000e+00,
        0.0000e+00, 1.1015e+02, 0.0000e+00, 0.0000e+00, 1.0000e+00, 9.6000e+01,
        0.0000e+00, 0.0000e+00, 1.1015e+02, 0.0000e+00, 0.0000e+00, 1.0000e+00,
        9.6000e+01, 0.0000e+00, 0.0000e+00, 1.1015e+02, 0.0000e+00, 0.0000e+00,
        1.0000e+00, 9.6000e+01, 0.0000e+00, 9.0949e-13, 1.1015e+02, 0.0000e+00,
        0.0000e+00, 1.0000e+00, 9.6000e+01, 0.0000e+00, 0.0000e+00, 1.1015e+02,
        0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
        0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
        0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
        9.6000e+01, 0.0000e+00, 0.0000e+00, 9.6000e+01, 0.0000e+00, 0.0000e+00,
        1.0000e+00, 9.6000e+01, 0.0000e+

In [16]:
features

,HT_MI_5_weight,HT_MI_5_mean,HT_MI_5_std,HT_MI_3_weight,HT_MI_3_mean,HT_MI_3_std,HT_MI_1_weight,HT_MI_1_mean,HT_MI_1_std,HT_MI_0.1_weight,...,HT_Hp_0.1_magnitude,HT_Hp_0.1_covariance,HT_Hp_0.1_pcc,HT_Hp_0.01_weight,HT_Hp_0.01_mean,HT_Hp_0.01_std,HT_Hp_0.01_radius,HT_Hp_0.01_magnitude,HT_Hp_0.01_covariance,HT_Hp_0.01_pcc
0,1.000000,54.000000,0.000000,1.000000,54.000000,0.000000,1.000000,54.000000,0.000000,1.000000,...,54.000000,0.000000,0.000000,1.000000,54.000000,0.000000,0.000000,54.000000,0.000000,0.000000
1,1.000000,96.000000,0.000000,1.000000,96.000000,0.000000,1.000000,96.000000,0.000000,1.000000,...,96.000000,0.000000,0.000000,1.000000,96.000000,0.000000,0.000000,96.000000,0.000000,0.000000
2,1.999384,91.498613,20.249998,1.999630,91.499168,20.249999,1.999877,91.499723,20.250000,1.999988,...,91.499972,0.000000,0.000000,1.999999,91.499997,20.250000,20.250000,91.499997,0.000000,0.000000
3,1.917074,64.954193,110.043709,1.949386,64.772624,110.175676,1.982836,64.590893,110.241738,1.998270,...,118.309953,0.000000,0.000000,1.000000,75.000000,0.000000,20.250000,118.309972,0.000000,0.000000
4,2.835460,71.671202,154.046528,2.899159,71.404677,155.705889,2.965658,71.135514,157.262148,2.996532,...,84.000000,0.000000,0.000000,1.000000,84.000000,0.000000,0.000000,84.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,64.390300,58.551943,1444.491701,71.245826,61.677086,3357.467390,85.315213,88.203451,31338.731422,155.374384,...,1456.919742,-1125.076738,-0.090714,76.962076,64.486558,4025.863982,40551.972963,1455.855829,-4200.761291,-0.329585
19996,65.320267,58.482256,1422.690064,72.199322,61.570754,3311.769647,86.296647,87.807104,30988.979572,156.371003,...,1456.913885,-1122.440902,-0.091074,77.961908,64.352049,3975.617564,40547.015494,1455.849877,-4188.699383,-0.330709
19997,66.272302,58.414622,1401.521302,73.167508,61.467283,3267.279452,87.283969,87.419780,30646.887504,157.368705,...,1456.908190,-1119.810009,-0.091428,78.961793,64.220947,3926.608927,40542.239555,1455.844088,-4176.699969,-0.331813
19998,67.238578,58.348966,1380.962832,74.145166,61.366571,3223.955371,88.275084,87.041194,30312.221632,158.367103,...,1456.902648,-1117.184313,-0.091777,79.961713,64.093124,3878.792948,40537.636403,1455.838455,-4164.762776,-0.332898


In [3]:
import torch

def binary_gradient(n):
    # Convert n to binary and reverse it
    binary = list(map(int, bin(n)[2:]))[::-1]

    # Convert each digit to a PyTorch tensor
    binary_tensors = [torch.tensor(float(b), requires_grad=True) for b in binary]

    # Compute the value of n as a function of the binary digits
    n_tensor = sum(b * 2**i for i, b in enumerate(binary_tensors))

    # Detach n_tensor from the computation graph and set requires_grad=True
    n_tensor = n_tensor.detach().requires_grad_(True)

    # Define the function
    f = n_tensor ** 2

    # Compute the gradient with respect to n
    f.backward()

    # The gradient with respect to n
    n_grad = n_tensor.grad.item()

    # The gradient with respect to each binary digit
    # binary_grads = [b.grad.item() for b in binary_tensors]
    binary_grads = [b.grad.item() if b.grad is not None else 0 for b in binary_tensors]
    print(binary_grads)

    return n_grad, binary_grads

# Use the function
n = 10
n_grad, binary_grads = binary_gradient(n)
print(f"The gradient with respect to {n} is {n_grad}")
print(f"The gradients with respect to each binary digit of {n} are {binary_grads}")

[0, 0, 0, 0]
The gradient with respect to 10 is 20.0
The gradients with respect to each binary digit of 10 are [0, 0, 0, 0]


In [1]:
# read the pcap file
import scapy.all as scapy
import pandas as pd

PCAP_PATH = "../../data/malicious/Port_Scanning_SmartTV.pcap"
# Read the pcap file
packets = scapy.rdpcap(PCAP_PATH)

# crop to first 500 packets
packets = packets[:500]

# save pcap file
scapy.wrpcap("../../data/malicious/Port_Scanning_SmartTV_500.pcap", packets)